# Notebook for the experiments
In this notebook are contained the following features:
* Re-implementation of GRAFF applied to [node classification](https://arxiv.org/abs/2206.10991),

The main tools that have been exploited are [PyTorch](https://pytorch.org/) (1.13.0), [PyTorch-Lightning](https://www.pytorchlightning.ai/index.html) (1.5.10), [Pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) (2.3.0) and [Weights & Biases](https://wandb.ai/)

### Requirements to run the notebook

In [1]:
# !pip install torch==1.13.0
# !pip install pytorch-lightning==1.5.10
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cu113.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu113.html
# !pip install torch-geometric
# !pip install wandb


## Importing the libraries

In [2]:
######## IMPORT EXTERNAL FILES ###########
import torch
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
import torch.nn as nn

import wandb

import torch_geometric
from torch_geometric.nn import MessagePassing
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import add_self_loops, degree, homophily
from torch_geometric.loader import DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger

from torch_geometric.datasets import WebKB, Planetoid, WikipediaNetwork
# WebKB: (Texas, Wisconsin, Cornell); Planetoid: (Citeseer, Pubmed, Cora); WikipediaNetwork: (Squirrel, Chameleon)

######### IMPORT INTERNAL FILES ###########

from GRAFF import *
from config import *

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### System configuration

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device == 'cuda' else 0

if wb:
    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


## PyTorch Lightning DataModule (Node classification)

In [4]:
class DataModuleNC(pl.LightningDataModule):

    def __init__(self,  dataset, mode, split, batch_size):

        self.mode = mode  # "hp" or "test"
        self.split = split
        self.batch_size = batch_size
        self.dataset = dataset

    def setup(self, stage=None):
        if stage == 'fit':
            if self.mode == 'hp':
                self.train_mask_pred = self.dataset[0].train_mask.bool()[
                    :, self.split]
            else:
                self.train_mask_pred = self.dataset[0].train_mask.bool(
                )[:, self.split] + self.dataset[0].val_mask.bool()[:, self.split]
        elif stage == 'test':
          
            if self.mode == 'hp':
                
                self.test_mask_pred = self.dataset[0].val_mask.bool()[
                    :, self.split]
            else:
                self.test_mask_pred = self.dataset[0].test_mask.bool()[
                    :, self.split]

    # In the transductive setting, we have available all the original graph's topology.
    def train_dataloader(self, *args, **kwargs):
        return DataLoader(self.dataset, batch_size = batch_size, shuffle = False)
    def val_dataloader(self, *args, **kwargs):
        return DataLoader(self.dataset, batch_size = batch_size, shuffle = False)
    

In [5]:
mode = 'hp'  # hp: Hyperparameter selection mode
dataM = DataModuleNC(dataset, split=0, mode=mode, batch_size=batch_size)
dataM.setup(stage='fit')
dataM.setup(stage='test') 

### PyTorch Lightning Callbacks

In [6]:

class Get_Metrics(Callback):

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

        # Compute the metrics
        train_loss = sum(
            pl_module.train_prop['loss']) / len(pl_module.train_prop['loss'])
        train_acc = sum(
            pl_module.train_prop['acc']) / len(pl_module.train_prop['acc'])
        test_loss = sum(
            pl_module.test_prop['loss']) / len(pl_module.test_prop['loss'])
        test_acc = sum(pl_module.test_prop['acc']) / \
            len(pl_module.test_prop['acc'])

        # Log the metrics
        pl_module.log(name='Loss on train', value=train_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Accuracy on train', value=train_acc,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Loss on test', value=test_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Accuracy on test', value=test_acc,
                      on_epoch=True, prog_bar=True, logger=True)

        # Re-initialize the metrics
        pl_module.train_prop['loss'] = []
        pl_module.train_prop['acc'] = []
        pl_module.test_prop['loss'] = []
        pl_module.test_prop['acc'] = []

## PyTorch Lightning Training Module (Node Classification)

In [7]:
class TrainingModule(pl.LightningModule):

    def __init__(self, model, loss_type, train_mask, test_mask, lr, wd):
        super().__init__()
        self.model = model
        self.loss_type = loss_type  # nll or ce
        self.lr = lr
        self.wd = wd
        
        self.train_prop = {'loss': [],
                           'acc': []}
        self.test_prop = {'loss': [],
                          'acc': []}
        self.test_mask = test_mask  # this can be referred to the test mask of the datamodule
        # this can be referred to the train mask of the datamodule
        self.train_mask = train_mask

    def training_step(self, batch, batch_idx):
        y = batch.y
        # print("Y BEFORE: ", y.shape)
        y = y[:batch_size]
        # print("Y AFTER: ", y.shape)

        out = self.model(batch)

        out = out[:batch_size]

        mask = self.train_mask

        if self.loss_type == 'nll':
            loss = F.nll_loss(F.log_softmax(out[mask], dim=1), y[mask])
        elif self.loss_type == 'ce':
            loss = F.cross_entropy(out[mask], y[mask])

        acc = evaluate(out, batch, mask)
        self.train_prop['loss'].append(loss)
        self.train_prop['acc'].append(acc)

        return loss

    def validation_step(self, batch, batch_idx):

        y = batch.y
        # print("Y BEFORE: (test)", y.shape)
        y = y[:batch_size]
        # print("Y AFTER: (test)", y.shape)

        ##### This line is to reduce the mask ######
        mask = self.test_mask
        ############################################

        out = self.model(batch)

        out = out[:batch_size]
        # print("OUT_SHAPE: ", out.shape)

        # print("TEST_MASK: ", mask.shape)

        if self.loss_type == 'nll':
            loss = F.nll_loss(F.log_softmax(
                out[mask], dim=1), y[mask])
        elif self.loss_type == 'ce':
            loss = F.cross_entropy(out[mask], y[mask])

        acc = evaluate(out, batch, mask)
        self.test_prop['loss'].append(loss)
        self.test_prop['acc'].append(acc)

        return loss

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.wd)
        return self.optimizer


def evaluate(out, data, mask=None):
    # print("OUTPUT IS DIM: ", out.shape)
    out = F.log_softmax(out, dim=1)
    # print("OUTPUT log_softmax shape is: ", out.shape)

    pred = out[mask].argmax(1)

    acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item(
    )

    return acc

In [8]:
# This mask contains the nodes for which we want to do the inference.
test_mask = dataM.test_mask_pred
train_mask = dataM.train_mask_pred

if mode != 'hp':
   model = PhysicsGNN_NC(dataset, hidden_dim, num_layers, step = step)
   pl_training_module = TrainingModule(model, loss_type, train_mask, test_mask, lr, wd)
 


### Hyperparameters Tuning

In [9]:
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        model = PhysicsGNN_NC(dataset, config.hidden_dim,
                              config.num_layers, step=config.step)
        pl_training_module = TrainingModule(
            model, config.loss_type, train_mask, test_mask, config.lr, config.wd)
        exp_name = "Sweep"
        wandb_logger = WandbLogger(
            project=project_name, name=exp_name, config=hyperparameters)
        trainer = trainer = pl.Trainer(
            max_epochs=epochs,  # maximum number of epochs.
            gpus=num_gpus,  # the number of gpus we have at our disposal.
            default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=5)],
            logger=wandb_logger
        )
        trainer.fit(model = pl_training_module, datamodule = dataM)

if mode == 'hp':

    import pprint

    pprint.pprint(sweep_config)

    sweep_id = wandb.sweep(sweep_config, project=project_name)

    wandb.agent(sweep_id, sweep_train, count=2)




{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Accuracy on test'},
 'parameters': {'hidden_dim': {'values': [32, 64, 128, 256, 512, 1024]},
                'loss_type': {'values': ['ce', 'nll']},
                'lr': {'values': [0.001, 0.0001, 1e-05, 1e-06]},
                'num_layers': {'values': [1, 2, 3, 4]},
                'step': {'values': [0.1, 0.2, 0.3, 0.4, 0.5]},
                'wd': {'values': [0, 0.01, 0.0001, 1e-06]}}}
Create sweep with ID: 3vc2rnm6
Sweep URL: https://wandb.ai/deepl_wizards/Link%20Prediction%20with%20PBGNN/sweeps/3vc2rnm6


wandb: Agent Starting Run: m6wd0ciw with config:
wandb: 	hidden_dim: 1024
wandb: 	loss_type: nll
wandb: 	lr: 0.001
wandb: 	num_layers: 3
wandb: 	step: 0.4
wandb: 	wd: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name  | Type          | Params
----------------------------------------
0 | model | PhysicsGNN_NC | 1.8 M 
----------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.000     Total estimated model params size (MB)
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/magistrale/Thesis/Link_Prediction_with_Physics-based_GNNs-/Sweep/version_None/checkpoints exists and is not empty.
  rank_zer

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_s

Epoch 1:  50%|█████     | 1/2 [00:00<00:00,  8.30it/s, loss=1.39, v_num=0ciw, Loss on train=1.670, Accuracy on train=0.115, Loss on test=1.480, Accuracy on test=0.314]

...

Epoch 1:  50%|█████     | 1/2 [00:00<00:00, 36.60it/s, loss=1.62, v_num=ggo7, Loss on train=1.620, Accuracy on train=0.149, Loss on test=1.610, Accuracy on test=0.229]

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


Epoch 99: 100%|██████████| 2/2 [00:00<00:00, 37.32it/s, loss=1.17, v_num=ggo7, Loss on train=1.130, Accuracy on train=0.713, Loss on test=1.340, Accuracy on test=0.525]


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


Accuracy on test,▁▂▄▅▆▆▆▇████████████████████████████████
Accuracy on train,▁▂▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████
Loss on test,██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
Loss on train,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Accuracy on test,0.52542
Accuracy on train,0.71264
Loss on test,1.34342
Loss on train,1.12798
epoch,99


In [ ]:


if wb:
    exp_name = "Node_class_lr: " + \
        str(hyperparameters['learning rate']) + \
        '_wd: ' + str(hyperparameters['weight decay'])
    description = ' initial tests'
    exp_name += description
    wandb_logger = WandbLogger(
        project=project_name, name=exp_name, config=hyperparameters)


trainer = trainer = pl.Trainer(
    max_epochs=epochs,  # maximum number of epochs.
    gpus=num_gpus,  # the number of gpus we have at our disposal.
    default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=30)],
    logger=wandb_logger if wb else None

)

In [ ]:
trainer.fit(model = pl_training_module, datamodule = dataM)
if wb:
    wandb.finish()